# DD2424 Project in Deep Learning in Data Science

## Imports

In [1]:
from functions import *
import pandas as pd
import numpy as np
import plotly.express as px
import tensorflow as tf

## Vanilla RNN

### Load and Preprocess Data

In [12]:

book_data = np.array(load_data('../Dataset/Training/vol1.txt'))
book_chars = np.unique(book_data)


char_to_ind = {ch:i for i,ch in enumerate(book_chars)}
ind_to_char = {i:ch for i,ch in enumerate(book_chars)}
k = book_chars.shape[0]
m = 100
eta = 0.1
seq_length = 25
sig = 0.01


### Train

In [13]:
rnn = RNN(m, k, eta, seq_length, sig)
h0 = np.zeros((m, 1))
epochs = 10
losslist = []
iteration = 0
smoothloss = 0
sentences = []
for i in range(epochs):
    rnn.hprev = np.zeros((m, 1))
    for e in range(0, book_data.shape[0]-seq_length-1, seq_length):
        X_chars = book_data[e:e+seq_length]
        Y_chars = book_data[e+1:e+seq_length+1]
        X = one_hot_encoding(X_chars, char_to_ind, k)
        Y = one_hot_encoding(Y_chars, char_to_ind, k)
        loss = rnn.adagrad(X, Y, h0)
        if smoothloss == 0:
            smoothloss = loss
        smoothloss = 0.999*smoothloss + 0.001*loss
     
        if iteration % 10000 == 0:
            print('Iteration: {}, Loss: {}'.format(iteration, smoothloss))
            y = rnn.synthetize(rnn.hprev, X[:, 0], 200)
            sentence = one_hot_decoding(y, ind_to_char)
            print(sentence)
            sentences.append(sentence)
            losslist.append(smoothloss)
        iteration += 1
        if iteration>400001:
            break
    if iteration>400001:
        break

Iteration: 0, Loss: [119.06065954]
ÃFhwS$7A0ïSCUNtO¶¢‰K@ -t0R0Oiz,‚‰©¿h«P¶sïhnlF_GD3f!†Ijm cl?9r¨fÃmfg1R¯‹#`68B‰GÃE6}‰Vª'¹!j,Za´‰k{‰¢*0B¼¡¹l<"¶®RBOMp"©‹S')cfpHyV)-¨O cgF«SWzDsI¿TaÃ»6¤Nnrz4jo&%7(2S'ÂP&Ngn¤`±!$†Ã`g4Zl9tm¼t#BdU‰bs 0*PQ}_V
